In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(0%)')
df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-0.028872,-0.130254,-3.184231e+07,-3.115075e+07,-7184.203318,...,-170775.886453,-1059.042606,-4.739468,-1.027193,-3584.970569,-5.649112,70.667599,70.667599,-1297.277948,0.006135
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.016772,-0.034656,-1.968379e+06,-1.924910e+06,-451.342112,...,-11502.243880,-168.146428,-1.410813,-1.207108,-222.196310,-1.681590,10.138780,10.138780,-79.398691,0.043215
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.015297,-0.094878,-6.059927e+07,-5.928166e+07,-13774.014902,...,-324959.759255,-3804.051175,-17.024036,-3.689647,-6861.392776,-20.291452,253.836024,253.836024,-2470.054721,0.011467
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.014625,NaN,-2.290124e+06,-2.239814e+06,-510.210093,...,-12199.928822,-901.413470,-0.189720,-1.009642,-253.366805,-0.226133,-3.169102,-3.169102,-92.619013,0.016390
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.006993,-0.050957,-3.997865e+07,-3.910761e+07,-9330.459280,...,-214438.078273,-4116.308632,-37.818819,-9.136488,-4606.916825,-45.077368,586.030400,586.030400,-1632.997165,0.033005


In [3]:
semiconductors_df = df[df['GICS Sub-Industry'] == 'Semiconductors']
semiconductors_df

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.248834,-1.041840,-2.352986e+08,-2.241057e+08,-51608.241955,...,-7.453941e+06,-3.020589e+03,-38.096485,-22.586870,-25725.305076,-45.408327,461.058008,461.058008,-9322.613928,0.007411
547,2019,ON SEMICONDUCTOR CORP,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.185307,-1.588235,-1.022506e+09,-8.437536e+08,-195077.858436,...,-1.638240e+08,-8.910291e+05,-287.356388,-187.908610,-96918.123466,-342.508573,3006.743867,3006.743867,-35008.897062,0.169190
550,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.152112,NaN,-7.536221e+08,-7.201719e+08,-165596.905315,...,-2.144053e+07,-1.582604e+04,-89.077183,-118.341067,-82515.872666,-106.173728,782.226632,782.226632,-29938.300593,0.004732
551,2019,WALTON ADVANCED ENG INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.150958,NaN,-3.394994e+07,-3.227687e+07,-7463.549213,...,-1.134506e+06,-7.183969e+02,-9.060618,-5.372733,-3713.028320,-10.799618,109.655005,109.655005,-1341.772299,0.013895
552,2019,DB HITEK CO LTD,"KOREA, REPUBLIC OF",Semiconductors,Manufacture of electrical machinery and appara...,-0.147224,-0.670886,-1.029421e+08,-9.831207e+07,-22601.109723,...,-2.987416e+06,-2.540078e+03,-12.016576,-3.823578,-11280.645693,-14.322912,151.830381,151.830381,-4090.525684,0.029233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13416,2010,ANALOG DEVICES INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.015666,-0.047080,-4.326103e+07,-3.929865e+07,-10000.183105,...,-2.902080e+06,-4.253137e+05,-137.163435,-89.694162,-4703.763185,-163.489152,1435.204970,1435.204970,-1583.564090,0.155052
13417,2010,MOTECH INDUSTRIES INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.013948,-0.101220,-1.860279e+07,-1.820192e+07,-4634.336274,...,-9.755134e+04,-4.120809e+03,-51.972762,-30.818624,-2209.637247,-61.947871,628.993935,628.993935,-748.237945,0.124131
13421,2010,SKYWORKS SOLUTIONS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.009606,-0.051817,-1.029631e+07,-7.979549e+06,-2218.041247,...,-2.026601e+06,-1.650811e+05,-53.238577,-34.813867,-994.003814,-63.456634,557.060055,557.060055,-311.899180,0.227818
13422,2010,ADVANCED MICRO DEVICES,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.008671,-0.100729,-5.630731e+07,-4.838061e+07,-13446.427984,...,-6.168118e+06,-1.000175e+06,-322.555994,-210.926400,-6026.362887,-384.464024,3375.053757,3375.053757,-1891.157188,0.245612


In [4]:
# Group the data by "Company Name" and count unique years for each group
company_years_count = semiconductors_df.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
filtered_df = semiconductors_df[semiconductors_df["Company Name"].isin(filtered_companies)]

In [5]:
new_df = filtered_df[["Company Name","Environmental Intensity (Sales)",
                      "Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
new_df = new_df.reset_index()
new_df = new_df.drop("index", axis=1)
new_df

,Company Name,Environmental Intensity (Sales),Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity
0,VANGUARD INTL SEMICONDUCTOR,-0.248834,-2.241057e+08,-51608.241955,-2.977838e+06,-6.980213e+05,-9361.183144,-3.020589e+03,-7.453941e+06,922.116017
1,UNITED MICROELECTRONICS CORP,-0.152112,-7.201719e+08,-165596.905315,-9.559184e+06,-2.240606e+06,-30028.483114,-1.582604e+04,-2.144053e+07,1564.453264
2,MACRONIX INTL CO LTD,-0.116371,-1.308858e+08,-29929.205341,-1.736436e+06,-4.069154e+05,-5443.332866,-3.737062e+03,-3.073505e+06,-31.350220
3,POWERTECH TECHNOLOGY INC,-0.059186,-1.270118e+08,-29010.724700,-1.682068e+06,-3.929992e+05,-5260.753996,-7.104034e+03,-2.496389e+06,-442.328492
4,ROHM CO LTD,-0.055297,-1.890929e+08,-44004.157255,-2.516677e+06,-5.891283e+05,-7928.270364,-1.870812e+03,-6.755350e+06,1512.288260
...,...,...,...,...,...,...,...,...,...,...
95,POWERTECH TECHNOLOGY INC,-0.054307,-6.886690e+07,-16207.461695,-9.134439e+05,-2.134952e+05,-2908.373205,-4.004698e+03,-3.754074e+05,1222.542083
96,ROHM CO LTD,-0.054161,-2.111819e+08,-64632.027555,-2.820921e+06,-6.617531e+05,-10212.662682,-1.503062e+03,-9.454682e+06,35058.080097
97,TEXAS INSTRUMENTS INC,-0.048182,-6.154968e+08,-140612.361583,-8.077119e+06,-1.892487e+06,-25419.492136,-2.150978e+06,-4.512573e+07,-3164.342910
98,FIRST SOLAR INC,-0.036372,-8.337028e+07,-19996.596265,-1.095669e+06,-2.555728e+05,-3549.697078,-3.908075e+05,-7.159446e+06,2637.530191


In [6]:
with_rev = filtered_df.copy()
with_rev

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.248834,-1.041840,-2.352986e+08,-2.241057e+08,-51608.241955,...,-7.453941e+06,-3.020589e+03,-38.096485,-22.586870,-25725.305076,-45.408327,461.058008,461.058008,-9322.613928,0.007411
550,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.152112,NaN,-7.536221e+08,-7.201719e+08,-165596.905315,...,-2.144053e+07,-1.582604e+04,-89.077183,-118.341067,-82515.872666,-106.173728,782.226632,782.226632,-29938.300593,0.004732
559,2019,MACRONIX INTL CO LTD,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.116371,-1.344026,-1.361418e+08,-1.308858e+08,-29929.205341,...,-3.073505e+06,-3.737062e+03,-7.475227,-27.944598,-14913.868588,-8.909944,-15.675110,-15.675110,-5435.764881,0.005121
581,2019,POWERTECH TECHNOLOGY INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.059186,-0.462201,-1.316251e+08,-1.270118e+08,-29010.724700,...,-2.496389e+06,-7.104034e+03,-1.261510,-53.131708,-14424.176811,-1.503631,-221.164246,-221.164246,-5259.476832,0.012212
582,2019,ROHM CO LTD,JAPAN,Semiconductors,Manufacture of electrical machinery and appara...,-0.055297,-0.386256,-1.990064e+08,-1.890929e+08,-44004.157255,...,-6.755350e+06,-1.870812e+03,-51.607215,-2.613648,-21937.286372,-61.512165,756.144130,756.144130,-7876.022767,0.001780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13397,2010,POWERTECH TECHNOLOGY INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.054307,-0.243988,-7.039114e+07,-6.886690e+07,-16207.461695,...,-3.754074e+05,-4.004698e+03,-50.508348,-29.950261,-7999.229809,-60.202393,611.271042,611.271042,-2857.238111,0.036247
13398,2010,ROHM CO LTD,JAPAN,Semiconductors,Manufacture of electrical machinery and appara...,-0.054161,-0.966493,-2.241605e+08,-2.111819e+08,-64632.027555,...,-9.454682e+06,-1.503062e+03,-1186.444660,-2.106374,-30913.418063,-1414.158460,17529.040049,17529.040049,-9011.495716,0.000145
13402,2010,TEXAS INSTRUMENTS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.048182,-0.152830,-6.729123e+08,-6.154968e+08,-140612.361583,...,-4.512573e+07,-2.150978e+06,-95.499410,-453.567089,-69513.186616,-113.828570,-1582.171455,-1582.171455,-25322.842554,0.025526
13404,2010,FIRST SOLAR INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.036372,-0.126067,-9.229269e+07,-8.337028e+07,-19996.596265,...,-7.159446e+06,-3.908075e+05,-126.035203,-82.417169,-9726.011236,-150.225084,1318.765096,1318.765096,-3422.104267,0.074137


In [20]:
with_rev["Revenue"] = with_rev["Total Environmental Cost"]/with_rev["Environmental Intensity (Sales)"]
with_rev

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.248834,-1.041840,-2.352986e+08,-2.241057e+08,-51608.241955,...,-3.020589e+03,-38.096485,-22.586870,-25725.305076,-45.408327,461.058008,461.058008,-9322.613928,0.007411,9.456030e+08
550,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.152112,NaN,-7.536221e+08,-7.201719e+08,-165596.905315,...,-1.582604e+04,-89.077183,-118.341067,-82515.872666,-106.173728,782.226632,782.226632,-29938.300593,0.004732,4.954381e+09
559,2019,MACRONIX INTL CO LTD,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.116371,-1.344026,-1.361418e+08,-1.308858e+08,-29929.205341,...,-3.737062e+03,-7.475227,-27.944598,-14913.868588,-8.909944,-15.675110,-15.675110,-5435.764881,0.005121,1.169897e+09
581,2019,POWERTECH TECHNOLOGY INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.059186,-0.462201,-1.316251e+08,-1.270118e+08,-29010.724700,...,-7.104034e+03,-1.261510,-53.131708,-14424.176811,-1.503631,-221.164246,-221.164246,-5259.476832,0.012212,2.223936e+09
582,2019,ROHM CO LTD,JAPAN,Semiconductors,Manufacture of electrical machinery and appara...,-0.055297,-0.386256,-1.990064e+08,-1.890929e+08,-44004.157255,...,-1.870812e+03,-51.607215,-2.613648,-21937.286372,-61.512165,756.144130,756.144130,-7876.022767,0.001780,3.598881e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13397,2010,POWERTECH TECHNOLOGY INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.054307,-0.243988,-7.039114e+07,-6.886690e+07,-16207.461695,...,-4.004698e+03,-50.508348,-29.950261,-7999.229809,-60.202393,611.271042,611.271042,-2857.238111,0.036247,1.296179e+09
13398,2010,ROHM CO LTD,JAPAN,Semiconductors,Manufacture of electrical machinery and appara...,-0.054161,-0.966493,-2.241605e+08,-2.111819e+08,-64632.027555,...,-1.503062e+03,-1186.444660,-2.106374,-30913.418063,-1414.158460,17529.040049,17529.040049,-9011.495716,0.000145,4.138743e+09
13402,2010,TEXAS INSTRUMENTS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.048182,-0.152830,-6.729123e+08,-6.154968e+08,-140612.361583,...,-2.150978e+06,-95.499410,-453.567089,-69513.186616,-113.828570,-1582.171455,-1582.171455,-25322.842554,0.025526,1.396600e+10
13404,2010,FIRST SOLAR INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.036372,-0.126067,-9.229269e+07,-8.337028e+07,-19996.596265,...,-3.908075e+05,-126.035203,-82.417169,-9726.011236,-150.225084,1318.765096,1318.765096,-3422.104267,0.074137,2.537459e+09


In [58]:
with_rev_un = with_rev[["Company Name","Revenue",
                      "Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
with_rev_un = with_rev_un.reset_index()
with_rev_un = with_rev_un.drop("index", axis=1)
with_rev_un

,Company Name,Revenue,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity
0,VANGUARD INTL SEMICONDUCTOR,9.456030e+08,-2.241057e+08,-51608.241955,-2.977838e+06,-6.980213e+05,-9361.183144,-3.020589e+03,-7.453941e+06,922.116017
1,UNITED MICROELECTRONICS CORP,4.954381e+09,-7.201719e+08,-165596.905315,-9.559184e+06,-2.240606e+06,-30028.483114,-1.582604e+04,-2.144053e+07,1564.453264
2,MACRONIX INTL CO LTD,1.169897e+09,-1.308858e+08,-29929.205341,-1.736436e+06,-4.069154e+05,-5443.332866,-3.737062e+03,-3.073505e+06,-31.350220
3,POWERTECH TECHNOLOGY INC,2.223936e+09,-1.270118e+08,-29010.724700,-1.682068e+06,-3.929992e+05,-5260.753996,-7.104034e+03,-2.496389e+06,-442.328492
4,ROHM CO LTD,3.598881e+09,-1.890929e+08,-44004.157255,-2.516677e+06,-5.891283e+05,-7928.270364,-1.870812e+03,-6.755350e+06,1512.288260
...,...,...,...,...,...,...,...,...,...,...
95,POWERTECH TECHNOLOGY INC,1.296179e+09,-6.886690e+07,-16207.461695,-9.134439e+05,-2.134952e+05,-2908.373205,-4.004698e+03,-3.754074e+05,1222.542083
96,ROHM CO LTD,4.138743e+09,-2.111819e+08,-64632.027555,-2.820921e+06,-6.617531e+05,-10212.662682,-1.503062e+03,-9.454682e+06,35058.080097
97,TEXAS INSTRUMENTS INC,1.396600e+10,-6.154968e+08,-140612.361583,-8.077119e+06,-1.892487e+06,-25419.492136,-2.150978e+06,-4.512573e+07,-3164.342910
98,FIRST SOLAR INC,2.537459e+09,-8.337028e+07,-19996.596265,-1.095669e+06,-2.555728e+05,-3549.697078,-3.908075e+05,-7.159446e+06,2637.530191


In [21]:
with_rev_sdg = with_rev[["Company Name","Revenue",
                         "SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]]
with_rev_sdg = with_rev_sdg.reset_index()
with_rev_sdg = with_rev_sdg.drop("index", axis=1)
with_rev_sdg

,Company Name,Revenue,SDG 1.5,SDG 2.1,SDG 2.2,SDG 2.3,SDG 2.4,SDG 3.3,SDG 3.4,SDG 3.9,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5
0,VANGUARD INTL SEMICONDUCTOR,9.456030e+08,-1.002095e+08,-5.847749e+07,-5.845129e+07,-7.444594e+05,-7.444594e+05,-9.427031e+06,2.475795e+05,-7.264702e+02,-7.453941e+06,-3.020589e+03,-38.096485,-22.586870,-25725.305076,-45.408327,461.058008,461.058008,-9322.613928
1,UNITED MICROELECTRONICS CORP,4.954381e+09,-3.216582e+08,-1.874539e+08,-1.873692e+08,-2.389796e+06,-2.389796e+06,-3.021503e+07,-5.748687e+05,-3.806259e+03,-2.144053e+07,-1.582604e+04,-89.077183,-118.341067,-82515.872666,-106.173728,782.226632,782.226632,-29938.300593
2,MACRONIX INTL CO LTD,1.169897e+09,-5.841251e+07,-3.393598e+07,-3.392041e+07,-4.341091e+05,-4.341091e+05,-5.468332e+06,-4.378029e+05,-8.987866e+02,-3.073505e+06,-3.737062e+03,-7.475227,-27.944598,-14913.868588,-8.909944,-15.675110,-15.675110,-5435.764881
3,POWERTECH TECHNOLOGY INC,2.223936e+09,-5.641273e+07,-3.270895e+07,-3.268740e+07,-4.205170e+05,-4.205170e+05,-5.268780e+06,-1.180796e+06,-1.708564e+03,-2.496389e+06,-7.104034e+03,-1.261510,-53.131708,-14424.176811,-1.503631,-221.164246,-221.164246,-5259.476832
4,ROHM CO LTD,3.598881e+09,-8.458654e+07,-4.963654e+07,-4.960897e+07,-6.291691e+05,-6.291691e+05,-8.005496e+06,8.752914e+05,-1.339181e+02,-6.755350e+06,-1.870812e+03,-51.607215,-2.613648,-21937.286372,-61.512165,756.144130,756.144130,-7876.022767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,POWERTECH TECHNOLOGY INC,1.296179e+09,-3.065681e+07,-1.810421e+07,-1.809287e+07,-2.283610e+05,-2.283610e+05,-2.921477e+06,2.310946e+05,-9.631548e+02,-3.754074e+05,-4.004698e+03,-50.508348,-29.950261,-7999.229809,-60.202393,611.271042,611.271042,-2857.238111
96,ROHM CO LTD,4.138743e+09,-9.534296e+07,-6.559387e+07,-6.555324e+07,-7.052303e+05,-7.052303e+05,-1.072890e+07,2.393270e+07,-1.075935e+02,-9.454682e+06,-1.503062e+03,-1186.444660,-2.106374,-30913.418063,-1414.158460,17529.040049,17529.040049,-9011.495716
97,TEXAS INSTRUMENTS INC,1.396600e+10,-2.716549e+08,-1.576373e+08,-1.575638e+08,-2.019280e+06,-2.019280e+06,-2.539795e+07,-7.997843e+06,-1.246510e+06,-4.512573e+07,-2.150978e+06,-95.499410,-453.567089,-69513.186616,-113.828570,-1582.171455,-1582.171455,-25322.842554
98,FIRST SOLAR INC,2.537459e+09,-3.670745e+07,-2.196744e+07,-2.195134e+07,-2.739173e+05,-2.739173e+05,-3.549037e+06,2.180161e+05,-2.264763e+05,-7.159446e+06,-3.908075e+05,-126.035203,-82.417169,-9726.011236,-150.225084,1318.765096,1318.765096,-3422.104267


In [59]:
X = with_rev_un[["Working Capacity", "Fish Production Capacity", "Crop Production Capacity",
                      "Meat Production Capacity", "Biodiversity", "Abiotic Resources",
                      "Water production capacity (Drinking water & Irrigation Water)", "Wood Production Capacity"]]
y = np.log(with_rev_un[["Revenue"]])
X = sm.add_constant(X)

In [56]:
X_sdg = with_rev_sdg[["SDG 1.5", "SDG 2.1", "SDG 2.2", "SDG 2.3", "SDG 2.4", "SDG 3.3", 
                         "SDG 3.4", "SDG 3.9", "SDG 6", "SDG 12.2", "SDG 14.1", "SDG 14.2", 
                         "SDG 14.3", "SDG 14.c", "SDG 15.1", "SDG 15.2", "SDG 15.5"]]
y_sdg = np.log(with_rev_sdg[["Revenue"]])
X_sdg = sm.add_constant(X_sdg)

In [26]:
X_sdg

,const,SDG 1.5,SDG 2.1,SDG 2.2,SDG 2.3,SDG 2.4,SDG 3.3,SDG 3.4,SDG 3.9,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5
0,1.0,-1.002095e+08,-5.847749e+07,-5.845129e+07,-7.444594e+05,-7.444594e+05,-9.427031e+06,2.475795e+05,-7.264702e+02,-7.453941e+06,-3.020589e+03,-38.096485,-22.586870,-25725.305076,-45.408327,461.058008,461.058008,-9322.613928
1,1.0,-3.216582e+08,-1.874539e+08,-1.873692e+08,-2.389796e+06,-2.389796e+06,-3.021503e+07,-5.748687e+05,-3.806259e+03,-2.144053e+07,-1.582604e+04,-89.077183,-118.341067,-82515.872666,-106.173728,782.226632,782.226632,-29938.300593
2,1.0,-5.841251e+07,-3.393598e+07,-3.392041e+07,-4.341091e+05,-4.341091e+05,-5.468332e+06,-4.378029e+05,-8.987866e+02,-3.073505e+06,-3.737062e+03,-7.475227,-27.944598,-14913.868588,-8.909944,-15.675110,-15.675110,-5435.764881
3,1.0,-5.641273e+07,-3.270895e+07,-3.268740e+07,-4.205170e+05,-4.205170e+05,-5.268780e+06,-1.180796e+06,-1.708564e+03,-2.496389e+06,-7.104034e+03,-1.261510,-53.131708,-14424.176811,-1.503631,-221.164246,-221.164246,-5259.476832
4,1.0,-8.458654e+07,-4.963654e+07,-4.960897e+07,-6.291691e+05,-6.291691e+05,-8.005496e+06,8.752914e+05,-1.339181e+02,-6.755350e+06,-1.870812e+03,-51.607215,-2.613648,-21937.286372,-61.512165,756.144130,756.144130,-7876.022767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,-3.065681e+07,-1.810421e+07,-1.809287e+07,-2.283610e+05,-2.283610e+05,-2.921477e+06,2.310946e+05,-9.631548e+02,-3.754074e+05,-4.004698e+03,-50.508348,-29.950261,-7999.229809,-60.202393,611.271042,611.271042,-2857.238111
96,1.0,-9.534296e+07,-6.559387e+07,-6.555324e+07,-7.052303e+05,-7.052303e+05,-1.072890e+07,2.393270e+07,-1.075935e+02,-9.454682e+06,-1.503062e+03,-1186.444660,-2.106374,-30913.418063,-1414.158460,17529.040049,17529.040049,-9011.495716
97,1.0,-2.716549e+08,-1.576373e+08,-1.575638e+08,-2.019280e+06,-2.019280e+06,-2.539795e+07,-7.997843e+06,-1.246510e+06,-4.512573e+07,-2.150978e+06,-95.499410,-453.567089,-69513.186616,-113.828570,-1582.171455,-1582.171455,-25322.842554
98,1.0,-3.670745e+07,-2.196744e+07,-2.195134e+07,-2.739173e+05,-2.739173e+05,-3.549037e+06,2.180161e+05,-2.264763e+05,-7.159446e+06,-3.908075e+05,-126.035203,-82.417169,-9726.011236,-150.225084,1318.765096,1318.765096,-3422.104267


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_model.predict(X_test)

# Calculate the mean squared error (you can use other evaluation metrics as needed)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# You can also access feature importances
feature_importances = gb_model.feature_importances_
print("Feature Importances:")
for i, feature in enumerate(X.columns):
    print(f"{feature}: {feature_importances[i]:.4f}")

Mean Squared Error: 0.11
Feature Importances:
const: 0.0000
Working Capacity: 0.0462
Fish Production Capacity: 0.0843
Crop Production Capacity: 0.0265
Meat Production Capacity: 0.0070
Biodiversity: 0.0427
Abiotic Resources: 0.5826
Water production capacity (Drinking water & Irrigation Water): 0.1507
Wood Production Capacity: 0.0601


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_sdg, y_sdg, test_size=0.2, random_state=42)

# Create a Gradient Boosting Regressor model
gb_model_sdg = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_model_sdg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_model_sdg.predict(X_test)

# Calculate the mean squared error (you can use other evaluation metrics as needed)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# You can also access feature importances
feature_importances = gb_model_sdg.feature_importances_
print("Feature Importances:")
for i, feature in enumerate(X_sdg.columns):
    print(f"{feature}: {feature_importances[i]:.4f}")

c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error: 0.13
Feature Importances:
const: 0.0000
SDG 1.5: 0.0305
SDG 2.1: 0.0213
SDG 2.2: 0.0300
SDG 2.3: 0.0244
SDG 2.4: 0.0492
SDG 3.3: 0.0211
SDG 3.4: 0.0230
SDG 3.9: 0.1690
SDG 6: 0.1491
SDG 12.2: 0.0877
SDG 14.1: 0.0098
SDG 14.2: 0.3116
SDG 14.3: 0.0287
SDG 14.c: 0.0032
SDG 15.1: 0.0089
SDG 15.2: 0.0172
SDG 15.5: 0.0154


In [46]:

# Perform backward elimination using statsmodels
def backward_elimination(X, y, significance_level=0.05):
    num_features = X.shape[1]
    for i in range(num_features):
        model = sm.OLS(y, X).fit()
        max_p_value = max(model.pvalues)
        if max_p_value > significance_level:
            feature_to_remove = np.argmax(model.pvalues[1:]) + 1  # +1 to exclude the constant term
            X = np.delete(X, feature_to_remove, axis=1)
        else:
            break
    return X

X_optimal = backward_elimination(X, y)

# Split the data into training and testing sets (you may want to adjust the test_size)
X_train, X_test, y_train, y_test = train_test_split(X_optimal, y, test_size=0.2, random_state=42)

# Create a Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_model.predict(X_test)

# Calculate the mean squared error (you can use other evaluation metrics as needed)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# You can also access feature importances
feature_importances = gb_model.feature_importances_
print("Feature Importances:")
for i, feature in enumerate(X_optimal.columns):
    print(f"{feature}: {feature_importances[i]:.4f}")

Mean Squared Error: 0.11
Feature Importances:
const: 0.0000
Working Capacity: 0.0462
Fish Production Capacity: 0.0843
Crop Production Capacity: 0.0265
Meat Production Capacity: 0.0070
Biodiversity: 0.0427
Abiotic Resources: 0.5826
Water production capacity (Drinking water & Irrigation Water): 0.1507
Wood Production Capacity: 0.0601


c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [55]:
original_feature_names = X_sdg.columns.tolist()

# Perform backward elimination using statsmodels
def backward_elimination(X, y, significance_level=0.05):
    num_features = X.shape[1]
    selected_features = []  # To store the features that were not removed
    for i in range(num_features):
        model = sm.OLS(y, X).fit()
        max_p_value = max(model.pvalues)
        if max_p_value > significance_level:
            feature_to_remove = np.argmax(model.pvalues[1:]) + 1  # +1 to exclude the constant term
            X = np.delete(X, feature_to_remove, axis=1)
        else:
            # If the feature was not removed, add it to the list of selected features
            selected_features.append(original_feature_names[feature_to_remove - 1])  # -1 to adjust for the constant term
            break
    return X, selected_features

X_optimal, selected_features = backward_elimination(X_sdg, y_sdg)

# Split the data into training and testing sets (you may want to adjust the test_size)
X_train, X_test, y_train, y_test = train_test_split(X_optimal, y_sdg, test_size=0.2, random_state=42)

# Create a Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model to the training data
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_model.predict(X_test)

# Calculate the mean squared error (you can use other evaluation metrics as needed)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")

# You can also access feature importances
feature_importances = gb_model.feature_importances_
print("Feature Importances:")
for i, feature in enumerate(original_feature_names):
    print(f"{feature}: {feature_importances[i]:.4f}")

c:\Users\ianpe\mambaforge\envs\PythonBase\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error: 0.11
Feature Importances:
const: 0.0000
SDG 1.5: 0.0248
SDG 2.1: 0.0351
SDG 2.2: 0.0303
SDG 2.3: 0.0624
SDG 2.4: 0.2969
SDG 3.3: 0.1493
SDG 3.4: 0.3191
SDG 3.9: 0.0061
SDG 6: 0.0316
SDG 12.2: 0.0085
SDG 14.1: 0.0120
SDG 14.2: 0.0238


IndexError: index 13 is out of bounds for axis 0 with size 13

In [69]:
# Standardize the independent variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Create and fit the Ridge regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X_scaled, y)

# Print the coefficients (weights) of the Ridge model
coefficients = ridge_model.coef_
intercept = ridge_model.intercept_

print("Intercept:", intercept)

for i, feature in enumerate(X.columns):
    print(f"{feature}: {coefficients[0][i]:.4f}")

Intercept: [22.1270162]
const: 0.0000
Working Capacity: -0.1819
Fish Production Capacity: -0.1654
Crop Production Capacity: -0.2085
Meat Production Capacity: -0.1895
Biodiversity: -0.1820
Abiotic Resources: -0.2644
Water production capacity (Drinking water & Irrigation Water): -0.1171
Wood Production Capacity: 0.2347


In [71]:
# Standardize the independent variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_sdg)

# Create and fit the Ridge regression model
alpha = 1.0  # You can adjust the regularization strength (alpha) as needed
ridge_model = Ridge(alpha=alpha)
ridge_model.fit(X_scaled, y_sdg)

# Print the coefficients (weights) of the Ridge model
coefficients = ridge_model.coef_
intercept = ridge_model.intercept_

print("Intercept:", intercept)

for i, feature in enumerate(X_sdg.columns):
    print(f"{feature}: {coefficients[0][i]:.4f}")

Intercept: [22.1270162]
const: 0.0000
SDG 1.5: -0.1150
SDG 2.1: -0.1132
SDG 2.2: -0.1119
SDG 2.3: -0.1335
SDG 2.4: -0.1335
SDG 3.3: -0.1121
SDG 3.4: -0.0346
SDG 3.9: -0.3506
SDG 6: -0.1199
SDG 12.2: -0.3162
SDG 14.1: 0.0051
SDG 14.2: 0.4240
SDG 14.3: -0.1076
SDG 14.c: 0.0051
SDG 15.1: 0.1278
SDG 15.2: 0.1278
SDG 15.5: -0.1097
